In [1]:
import sys
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().resolve().parents[0]
sys.path.insert(0, str(PROJECT_ROOT))

In [2]:
from src.cohort import load_aki_cohort
from src.db import q

df_aki = load_aki_cohort()


In [3]:
df_aki["icu_los_days"] = (
    (df_aki["outtime"] - df_aki["intime"])
    .dt.total_seconds() / (3600 * 24)
)


In [4]:
df_rrt_proc = q("""
SELECT DISTINCT pe.icustay_id
FROM procedureevents_mv pe
JOIN d_items di ON pe.itemid = di.itemid
WHERE
    LOWER(di.label) LIKE '%hemodial%'
 OR LOWER(di.label) LIKE '%haemodial%'
 OR LOWER(di.label) LIKE '%crrt%'
 OR LOWER(di.label) LIKE '%dialysis%'
""")

df_rrt_icd = q("""
SELECT DISTINCT hadm_id
FROM procedures_icd
WHERE icd9_code IN ('3995','5498')
""")

df_aki["dialysis"] = (
    df_aki["icustay_id"].isin(df_rrt_proc["icustay_id"])
    | df_aki["hadm_id"].isin(df_rrt_icd["hadm_id"])
).astype(int)



In [5]:
df_dopamine = q("""
SELECT
  ie.icustay_id,
  ie.starttime
FROM inputevents_mv ie
JOIN d_items di
  ON ie.itemid = di.itemid
WHERE LOWER(di.label) LIKE '%dopamine%'
""")


In [6]:
df_dopa = df_dopamine.merge(
    df_aki[["icustay_id", "intime"]],
    on="icustay_id",
    how="inner"
)


In [7]:
df_dopa["hours_since_icu"] = (
    (df_dopa["starttime"] - df_dopa["intime"])
    .dt.total_seconds() / 3600
)


In [8]:
early_dopa_ids = (
    df_dopa.loc[
        (df_dopa["hours_since_icu"] >= 0)
        & (df_dopa["hours_since_icu"] <= 24),
        "icustay_id"
    ]
    .unique()
)

df_aki["early_dopamine"] = df_aki["icustay_id"].isin(early_dopa_ids).astype(int)


In [9]:
df_aki["early_dopamine"].value_counts(normalize=True)


early_dopamine
0    0.967859
1    0.032141
Name: proportion, dtype: float64

In [10]:
df_aki.groupby("early_dopamine").agg(
    n=("early_dopamine", "count"),
    mortality=("hospital_mortality", "mean"),
    icu_los_median=("icu_los_days", "median"),
    dialysis_rate=("dialysis", "mean")
)


,n,mortality,icu_los_median,dialysis_rate
early_dopamine,,,,
0,10148,0.213047,3.006429,0.130075
1,337,0.394659,3.801389,0.195846
